# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 


In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
parquet_files


['../../05_src/data/prices\\ACN\\ACN_2001\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2001\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2002\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2003\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2004\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2004\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2005\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2005\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2006\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2006\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2007\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2007\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2008\\part.0.parquet',
 '../../05_src/data/prices\\ACN\\ACN_2008\\part.1.parquet',
 '../../05_src/data/prices\\ACN\\ACN_200

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Write your code below.
import pandas as pd
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
dd_shift = (
    dd_px
        .groupby('ticker', group_keys=False)
        .apply(
            lambda x: x.sort_values('Date', ascending = True)
                       .assign(Close_lag_1 = x['Close'].shift(1),
                               Adj_Close_lag_1 = x['Adj Close'].shift(1)
                               ), 
            meta = pd.DataFrame(data ={'Date': 'datetime64[ns]',
                    'Open': 'f8',
                    'High': 'f8',
                    'Low': 'f8',
                    'Close': 'f8',
                    'Adj Close': 'f8',
                    'Volume': 'i8',
                    'source': 'object',
                    'Year': 'int32',
                    'Close_lag_1': 'f8',
                    'Adj_Close_lag_1': 'f8',},
                    index = pd.Index([], dtype=pd.StringDtype(), name='ticker'))
    ))

dd_feat = dd_shift.assign(
    returns = lambda x: (x['Close']/x['Close_lag_1']) - 1,
    hi_lo_range = lambda x: x['High'] - x['Low'],
)

dd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range
npartitions=90,,,,,,,,,,,,,
ACN,object,object,object,object,object,object,object,object,object,object,object,object,object
ALDX,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [5]:
# Write your code below.
# converting to pandas dataframe
pdf = dd_feat.compute()
# adding mins-period = 1 so will work for rows that do not reach the 10 count as yet
pdf['moving_average_10_days'] = pdf['returns'].rolling(10, 1).mean()
pdf.head(100)


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close_lag_1,returns,hi_lo_range,moving_average_10_days
ticker,,,,,,,,,,,,,,
ACN,2001-07-19,15.100000,15.290000,15.000000,15.170000,11.404394,34994300.0,ACN.csv,2001,NaN,NaN,NaN,0.290000,NaN
ACN,2001-07-20,15.050000,15.050000,14.800000,15.010000,11.284108,9238500.0,ACN.csv,2001,15.170000,11.404394,-0.010547,0.250000,-0.010547
ACN,2001-07-23,15.000000,15.010000,14.550000,15.000000,11.276587,7501000.0,ACN.csv,2001,15.010000,11.284108,-0.000666,0.460000,-0.005607
ACN,2001-07-24,14.950000,14.970000,14.700000,14.860000,11.171341,3537300.0,ACN.csv,2001,15.000000,11.276587,-0.009333,0.270000,-0.006849
ACN,2001-07-25,14.700000,14.950000,14.650000,14.950000,11.238999,4208100.0,ACN.csv,2001,14.860000,11.171341,0.006057,0.300000,-0.003623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ACN,2001-12-07,25.900000,25.950001,24.799999,24.860001,18.689070,1115400.0,ACN.csv,2001,26.250000,19.734028,-0.052952,1.150002,0.020255
ACN,2001-12-10,25.000000,26.200001,24.500000,24.600000,18.493610,1732100.0,ACN.csv,2001,24.860001,18.689070,-0.010459,1.700001,0.014586
ACN,2001-12-11,24.549999,24.879999,24.500000,24.760000,18.613895,1713200.0,ACN.csv,2001,24.600000,18.493610,0.006504,0.379999,0.010585


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

If calculating the average return could be done in Dask, it probably wouldn't be necessary to convert to pandas. However, pandas display the output with data much faster than Dask's compute().So in terms of usability / testing pandas would be preferred. However, if we had really large dataset (that couldn't fit in memory) or reading over a network, Dask would have been preferred. 

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.